In [110]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [111]:
data = pd.read_csv('connect4data.csv', index_col=False)
data.head()

,outcome,moves,board,-,-.1,-.2,-.3,-.4,-.5,-.6,...,-.80,-.81,-.82,policy,-.83,-.84,-.85,-.86,-.87,-.88
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0.1188,0.1228,0.1520,0.2273,0.1353,0.1310,0.1128
1,-1,1,0,0,0,0,0,0,0,0,...,0,0,0,0.1183,0.1418,0.1450,0.1693,0.1742,0.1481,0.1033
2,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0.1304,0.1740,0.1682,0.1829,0.1002,0.1486,0.0957
3,-1,2,0,0,0,0,0,0,0,0,...,0,0,0,0.1594,0.1316,0.1507,0.2138,0.1661,0.0902,0.0882
4,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0.1197,0.0982,0.1828,0.2243,0.1225,0.1206,0.1319


In [112]:
ENTRIES = data.shape[0]

In [113]:
states = np.array(data[["board", "-"] + [f"-.{i}" for i in range(1, 83)]])
policies = np.array(data[["policy"] + [f"-.{i}" for i in range(83, 89)]])
states[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [114]:
states = states.reshape(ENTRIES, 6, 7, 2)  # reshape to 6x7x2 for cnn

In [115]:
data = list(zip(states, policies))

In [116]:
# shuffle data
np.random.shuffle(data)
# split into train and test
train_data = data[:int(ENTRIES*0.8)]
test_data = data[int(ENTRIES*0.8):]

In [117]:
# define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu", input_shape=(6, 7, 2)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(7, activation="softmax")
])

In [118]:
# compile model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [119]:
# train model
xs = np.array([x for x, _ in train_data])
ys = np.array([y for _, y in train_data])
xs_test = np.array([x for x, _ in test_data])
ys_test = np.array([y for _, y in test_data])

In [120]:
model.fit(xs, ys, epochs=100, validation_data=(xs_test, ys_test))

Epoch 1/100
139/139 [==============================] - 2s 11ms/step - loss: 1.8186 - accuracy: 0.2369 - val_loss: 1.7081 - val_accuracy: 0.2970
Epoch 2/100
139/139 [==============================] - 1s 8ms/step - loss: 1.6237 - accuracy: 0.3204 - val_loss: 1.5434 - val_accuracy: 0.3546
Epoch 3/100
139/139 [==============================] - 1s 7ms/step - loss: 1.5160 - accuracy: 0.3760 - val_loss: 1.5063 - val_accuracy: 0.3798
Epoch 4/100
139/139 [==============================] - 1s 8ms/step - loss: 1.4626 - accuracy: 0.4228 - val_loss: 1.4457 - val_accuracy: 0.4239
Epoch 5/100
139/139 [==============================] - 1s 8ms/step - loss: 1.4252 - accuracy: 0.4543 - val_loss: 1.4451 - val_accuracy: 0.4464
Epoch 6/100
139/139 [==============================] - 1s 7ms/step - loss: 1.4065 - accuracy: 0.4793 - val_loss: 1.4259 - val_accuracy: 0.4446
Epoch 7/100
139/139 [==============================] - 1s 8ms/step - loss: 1.3874 - accuracy: 0.4885 - val_loss: 1.4186 - val_accuracy: 0.435

In [121]:
mock_board = np.zeros((6, 7, 2)) # looks like the starting position
model.predict(mock_board.reshape(1, 6, 7, 2))

array([[0.12579541, 0.14169508, 0.13677353, 0.14943083, 0.17090951,
        0.14900477, 0.12639084]], dtype=float32)

In [122]:
mock_board2 = np.zeros((6, 7, 2))  # looks like the starting position
mock_board2[5, 3, 0] = 1 # place a piece in column 4 (index 3)
model.predict(mock_board2.reshape(1, 6, 7, 2))

array([[0.14260529, 0.08581785, 0.13964482, 0.21713383, 0.16712706,
        0.13802135, 0.10964973]], dtype=float32)